In [ ]:
!pip install --upgrade transformers
!pip install --upgrade datasets
!pip install --upgrade huggingface_hub
!pip install evaluate
!pip install accelerate
!pip install ptflops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 45.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
from huggingface_hub import login
from transformers import AutoImageProcessor, AutoFeatureExtractor
from torchvision.transforms import (CenterCrop,
                                    Compose,
                                    Normalize,
                                    RandomHorizontalFlip,
                                    RandomResizedCrop,
                                    Resize,
                                    ToTensor)
from torch.utils.data import DataLoader
import torch
from transformers import ViTForImageClassification, PvtForImageClassification, SwinForImageClassification, CvtForImageClassification, LevitForImageClassification, ViTHybridForImageClassification, DeiTForImageClassification, DeiTForImageClassificationWithTeacher
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['image']]
    return examples

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return dict(accuracy=accuracy_score(predictions, labels))

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
val_ds = load_dataset('Tsomaros/Imagenet-1k_validation')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/27.3k [00:00<?, ?B/s]

validation-00000-of-00015.parquet:   0%|          | 0.00/495M [00:00<?, ?B/s]

validation-00001-of-00015.parquet:   0%|          | 0.00/497M [00:00<?, ?B/s]

validation-00002-of-00015.parquet:   0%|          | 0.00/467M [00:00<?, ?B/s]

...

validation-00011-of-00015.parquet:   0%|          | 0.00/414M [00:00<?, ?B/s]

validation-00012-of-00015.parquet:   0%|          | 0.00/443M [00:00<?, ?B/s]

validation-00013-of-00015.parquet:   0%|          | 0.00/438M [00:00<?, ?B/s]

validation-00014-of-00015.parquet:   0%|          | 0.00/478M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
model_name = 'microsoft/swin-large-patch4-window7-224' #@param ["google/vit-base-patch16-224","google/vit-large-patch16-224", "google/vit-hybrid-base-bit-384", "Zetatech/pvt-medium-224", "Zetatech/pvt-large-224", "microsoft/swin-base-patch4-window7-224", "microsoft/swin-large-patch4-window7-224", "microsoft/swin-large-patch4-window12-384", "microsoft/cvt-21", "microsoft/cvt-21-384", "microsoft/cvt-21-384-22k", "facebook/levit-256", "facebook/levit-384", "facebook/deit-base-patch16-224", "facebook/deit-base-patch16-384", "facebook/deit-base-distilled-patch16-224"]

if model_name == 'google/vit-base-patch16-224':
  model_id = 'vit-base-patch16-224'
  processor = AutoImageProcessor.from_pretrained(model_name)
  model = ViTForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,224,224)
  size = processor.size["height"]

elif model_name == 'google/vit-large-patch16-224':
  model_id = 'vit-large-patch16-224'
  processor = AutoImageProcessor.from_pretrained(model_name)
  model = ViTForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,224,224)
  size = processor.size["height"]

elif model_name == 'google/vit-hybrid-base-bit-384':
  model_id = 'vit-hybrid-base-bit-384'
  processor = AutoImageProcessor.from_pretrained(model_name)
  model = ViTHybridForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,384,384)
  size = processor.size["shortest_edge"]

elif model_name == 'Zetatech/pvt-medium-224':
  model_id = 'pvt-medium-224'
  processor = AutoImageProcessor.from_pretrained(model_name)
  model = PvtForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,224,224)
  size = processor.size["height"]

elif model_name == 'Zetatech/pvt-large-224':
  model_id = 'pvt-large-224'
  processor = AutoImageProcessor.from_pretrained(model_name)
  model = PvtForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,224,224)
  size = processor.size["height"]

elif model_name == 'microsoft/swin-base-patch4-window7-224':
  model_id = 'swin-base-patch4-window7-224'
  processor = AutoImageProcessor.from_pretrained(model_name)
  model = SwinForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,224,224)
  size = processor.size["height"]

elif model_name == 'microsoft/swin-large-patch4-window7-224':
  model_id = 'swin-large-patch4-window7-224'
  processor = AutoImageProcessor.from_pretrained(model_name)
  model = SwinForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,224,224)
  size = processor.size["height"]

elif model_name == 'microsoft/swin-large-patch4-window12-384':
  model_id = 'swin-large-patch4-window12-384'
  processor = AutoImageProcessor.from_pretrained(model_name)
  model = SwinForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,384,384)
  size = processor.size["height"]

elif model_name == 'microsoft/cvt-21':
  model_id = 'cvt-21'
  processor = AutoImageProcessor.from_pretrained(model_name)
  model = CvtForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,224,224)
  size = processor.size["shortest_edge"]

elif model_name == 'microsoft/cvt-21-384':
  model_id = 'cvt-21-384'
  processor = AutoImageProcessor.from_pretrained(model_name)
  model = CvtForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,384,384)
  size = processor.size["shortest_edge"]

elif model_name == 'microsoft/cvt-21-384-22k':
  model_id = 'cvt-21-384-22k'
  processor = AutoImageProcessor.from_pretrained(model_name)
  model = CvtForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,384,384)
  size = processor.size["shortest_edge"]

elif model_name == 'facebook/levit-256':
  model_id = 'levit-256'
  processor = AutoFeatureExtractor.from_pretrained(model_name)
  model = LevitForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,224,224)
  size = processor.crop_size["height"]

elif model_name == 'facebook/levit-384':
  model_id = 'levit-384'
  processor = AutoFeatureExtractor.from_pretrained(model_name)
  model = LevitForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,224,224)
  size = processor.crop_size["height"]

elif model_name == 'facebook/deit-base-patch16-224':
  model_id = 'deit-base-patch16-224'
  processor = AutoFeatureExtractor.from_pretrained(model_name)
  model = ViTForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,224,224)
  size = processor.size["height"]

elif model_name == 'facebook/deit-base-patch16-384':
  model_id = 'deit-base-patch16-384'
  processor = AutoFeatureExtractor.from_pretrained(model_name)
  model = ViTForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,224,224)
  size = processor.size["height"]

elif model_name == 'facebook/deit-base-distilled-patch16-224':
  model_id = 'deit-base-distilled-patch16-224'
  processor = AutoFeatureExtractor.from_pretrained(model_name)
  model = DeiTForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)
  image_res = (3,224,224)
  size = processor.size["height"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

image_mean, image_std = processor.image_mean, processor.image_std

normalize = Normalize(mean=image_mean, std=image_std)

_val_transforms = Compose(
    [
        Resize(size),
        CenterCrop(size),
        ToTensor(),
        normalize,
    ]
)

val_ds.set_transform(val_transforms)


In [ ]:
args = TrainingArguments(
    f'{model_id}',
    evaluation_strategy="epoch",
    per_device_eval_batch_size=16,
    logging_dir='logs',
    remove_unused_columns=False,
    do_eval=True
)

trainer = Trainer(
    model,
    args,
    eval_dataset=val_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  eval_validation_accuracy               =     0.8217
  eval_validation_loss                   =     1.4685
  eval_validation_model_preparation_time =     0.0333
  eval_validation_runtime                = 0:34:02.33
  eval_validation_samples_per_second     =     24.482
  eval_validation_steps_per_second       =       1.53


In [ ]:
from ptflops import get_model_complexity_info

macs, params = get_model_complexity_info(model, image_res, as_strings=False, print_per_layer_stat=False, verbose=False)
macs = round(int(macs) / 1000000000, 1)
params = round(int(params) / 1000000, 1)

print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Computational complexity:       49.4    
Number of parameters:           86.9    


In [ ]:
import json

filename = f'{model_id}' + '/eval_results.json'

with open(filename) as fp:
  dictObj = json.load(fp)

dictObj.update({"parameters": params,
                "GMacs": macs,
                "Device": torch.cuda.get_device_name(0),
                "Model Id": model_id})


In [ ]:
with open(filename, 'w') as json_file:
    json.dump(dictObj, json_file,
                        indent=4,
                        separators=(',',': '))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%mkdir /content/drive/MyDrive/imagenet-1k/{model_id}
%cp /content/{model_id}/eval_results.json /content/drive/MyDrive/imagenet-1k/{model_id}

Mounted at /content/drive
